In [1]:
!pip install datasets

In [2]:
import jax
import jax.numpy as jnp
from jax import grad, jit, random
from jax import lax
from jax.tree_util import tree_map
import numpy as np
from jax import grad, jit, random
from jax import lax
from jax.tree_util import tree_map
from transformers import GPT2Model, GPT2Tokenizer
import torch
import numpy as np
from datasets import load_dataset

In [3]:
# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model_gpt = GPT2Model.from_pretrained('gpt2')
vocab = tokenizer.get_vocab()
model_gpt.eval()

def break_word_gpt(text):
    tokens = tokenizer.tokenize(text)
    return tokens

def break_into_id(text):
    return tokenizer.encode(text, add_special_tokens=False)

def encode_word_gpt(word):
    token_ids = tokenizer.encode(word, add_special_tokens=False)
    token_tensor = torch.tensor([token_ids])
    with torch.no_grad():
        token_embeddings = model_gpt.wte(token_tensor)
    return token_embeddings

# Load the PTB text dataset
dataset = load_dataset('ptb_text_only')
train_data = dataset['train']
test_data = dataset['test']
print(dataset)

def read_batch_file(start_index, size, data):
    data_token = []
    for sentence_pair_index in range(start_index, start_index + size):
        if sentence_pair_index < len(data):
            data_token.append(break_word_gpt(data[sentence_pair_index]["sentence"]))
    return data_token
def make_data(no_token,data):
  data_tokens = read_batch_file(0, int(len(train_data)/1), train_data)
  encode_tokens , encode_tokens_test ,x , y, y_LLM = [],[],[],[],[]
  temp = np.zeros(2*768)
  for i in data_tokens:
      encode_tokens.append(encode_word_gpt(i)[0])
  for i in range(int(len(data)/1)):
      if len(data_tokens[i]) > no_token:
          y.append(encode_tokens[i][no_token])
          y_LLM.append(break_into_id([data_tokens[i][no_token]]))
  x = np.zeros(shape=(len(y),no_token // 2,2*768))
  count_index = 0
  for i in range(int(len(data)/1)):
      if len(data_tokens[i]) > no_token:
          for k in range(no_token):
              if k % 2 == 0:
                  x[count_index][int(k/2)][:768] = encode_tokens[i][k]
              else:
                  x[count_index][int(k/2)][768:] = encode_tokens[i][k]
          count_index += 1
  x = np.transpose(np.transpose(x))
  y = jnp.array(y)
  x = jnp.array(x)
  return (x, y, y_LLM)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

The repository for ptb_text_only contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ptb_text_only.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/42068 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3761 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3370 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 42068
    })
    test: Dataset({
        features: ['sentence'],
        num_rows: 3761
    })
    validation: Dataset({
        features: ['sentence'],
        num_rows: 3370
    })
})


In [4]:
x, y, LLM_y = make_data(4, train_data)
x_test, y_test, LLM_y_test = make_data(4, test_data)
print(x.shape, y.shape)
print(x_test.shape, y_test.shape)

(41358, 2, 1536) (41358, 768)
(3696, 2, 1536) (3696, 768)


In [5]:
# Initialize network parameters
def init_network_params(layer_sizes, key):
    keys = random.split(key, len(layer_sizes))
    return [(random.normal(k, (m, n)) * 0.01, random.normal(k, (n,)) * 0.01)
            for m, n, k in zip(layer_sizes[:-1], layer_sizes[1:], keys)]

# Initialize Adam optimizer state
def init_adam_state(params):
    return [(jnp.zeros_like(w), jnp.zeros_like(w), jnp.zeros_like(b), jnp.zeros_like(b)) for w, b in params]

# Adam optimizer update
def adam_update(params, grads, opt_state, t, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
    new_params = []
    new_opt_state = []

    for (w, b), (dw, db), (m_w, v_w, m_b, v_b) in zip(params, grads, opt_state):
        m_w = beta1 * m_w + (1 - beta1) * dw
        v_w = beta2 * v_w + (1 - beta2) * (dw ** 2)
        m_b = beta1 * m_b + (1 - beta1) * db
        v_b = beta2 * v_b + (1 - beta2) * (db ** 2)

        m_w_hat = m_w / (1 - beta1 ** t)
        v_w_hat = v_w / (1 - beta2 ** t)
        m_b_hat = m_b / (1 - beta1 ** t)
        v_b_hat = v_b / (1 - beta2 ** t)

        w = w - lr * m_w_hat / (jnp.sqrt(v_w_hat) + eps)
        b = b - lr * m_b_hat / (jnp.sqrt(v_b_hat) + eps)

        new_params.append((w, b))
        new_opt_state.append((m_w, v_w, m_b, v_b))

    return new_params, new_opt_state

# Activation functions
def relu(x):
    return jnp.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + jnp.exp(-x))

def tanh(x):
    return jnp.tanh(x)

def softmax(x):
    exps = jnp.exp(x - jnp.max(x))
    return exps / exps.sum(axis=-1, keepdims=True)

# Define the neural network forward pass with activation function indices
def forward_pass(params, x, activation_indices):
    activations_output = x
    for i, (w, b) in enumerate(params[:-1]):
        outputs = jnp.dot(activations_output, w) + b
        activations_output = apply_activation(outputs, activation_indices[i])
    final_w, final_b = params[-1]
    return apply_activation(jnp.dot(activations_output, final_w) + final_b,activation_indices[-1])
    # return jnp.dot(activations_output, final_w) + final_b

# Helper function to apply activation functions using lax.cond
def apply_activation(x, activation_index):
    return lax.switch(activation_index, [relu, sigmoid, tanh, softmax], x)

# Loss function: Mean Squared Error
def process_tokens_single_example(params1, params2, x, activation_indices1, activation_indices2):
  for layer_idx in range(int(np.log2(x.shape[0]))):
    if layer_idx == 0:
      temp = forward_pass(params1, x, activation_indices1)
    else:
      temp = forward_pass(params1, input_next_layer, activation_indices1)
    input_next_layer = jnp.zeros(shape = (temp.shape[0] // 2 , 2*768))
    for k in range(temp.shape[0]):
      if k % 2 == 0:
          input_next_layer = input_next_layer.at[k // 2,:768].set(temp[k])
      else:
        input_next_layer = input_next_layer.at[k // 2 ,768:].set(temp[k])
  EO = forward_pass(params1, input_next_layer, activation_indices2)
  return forward_pass(params2, EO, activation_indices2)
def loss_single_example(params1, params2, x, y, activation_indices1, activation_indices2):
    next_predicted_token_embedding = process_tokens_single_example(params1, params2, x, activation_indices1, activation_indices2)
    return jnp.sum(jnp.abs(next_predicted_token_embedding - y))

loss_vectorized = jax.vmap(fun = loss_single_example, in_axes=(None,None,0,0,None,None))
process_tokens_vectorized = jax.vmap(fun = process_tokens_single_example, in_axes=(None, None, 0, None, None))


def loss(params1, params2,  x, y, activation_indices1, activation_indices2):
    return jnp.sum(loss_vectorized(params1, params2, x, y, activation_indices1, activation_indices2))

# Example usage
layer_sizes1 = [2*768,  768]  # Depth and width of the first network
layer_sizes2 = [768,  768]   # Depth and width of the second network

activation_indices1 = [2]  # Activation functions for each layer of the first network: 0=ReLU, 1=Sigmoid, 2=Tanh, 3=Softmax
activation_indices2 = [2]  # Activation functions for each layer of the second network

In [6]:
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LayerNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
with open('params1_4_tokens', 'rb') as f:
    params1 = pickle.load(f)
with open('params2_4_tokens', 'rb') as f:
    params2 = pickle.load(f)
perdector = load_model('perdector_4.h5')

In [7]:
print(loss(params1, params2, x, y, activation_indices1, activation_indices2)/(x.shape[0]*768))
print(loss(params1, params2, x_test, y_test, activation_indices1, activation_indices2)/(x_test.shape[0]*768))

0.052321017
0.051815182


In [8]:
perdector.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use categorical crossentropy for multi-class classification
              metrics=['accuracy'])

In [9]:
LLM_x_test = process_tokens_vectorized(params1, params2, x_test, activation_indices1, activation_indices2)
LLM_x_test = np.array(jnp.transpose(LLM_x_test , (1,0,2))[0])
print(LLM_x_test.shape)
LLM_y_test = np.array(LLM_y_test)
LLM_y_test =  LLM_y_test.reshape(-1)
print(type(LLM_x_test),type(LLM_y_test))
print(LLM_x_test.shape,LLM_y_test.shape)

(3696, 768)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(3696, 768) (3696,)


In [10]:
process_tokens_vectorized = jax.vmap(fun = process_tokens_single_example, in_axes=(None, None,  0, None, None))
LLM_x = process_tokens_vectorized(params1, params2, x, activation_indices1, activation_indices2)
LLM_x = np.array(jnp.transpose(LLM_x , (1,0,2))[0])
print(LLM_x.shape)
LLM_y = np.array(LLM_y)
LLM_y =  LLM_y.reshape(-1)
print(type(LLM_x),type(LLM_y))
print(LLM_x.shape,LLM_y.shape)
perdector.evaluate(LLM_x_test,LLM_y_test)

(41358, 768)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(41358, 768) (41358,)
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6472 - loss: 1.9660


[1.9330741167068481, 0.6463744640350342]

In [11]:
perdector.evaluate(LLM_x,LLM_y)

1293/1293 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6448 - loss: 1.9368


[1.9218459129333496, 0.6484355926513672]